In [1]:
import yfinance as yf
from datetime import timedelta, timezone, time, datetime, date
import matplotlib.pyplot as plt
import pytz
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual, FloatSlider, interactive_output, Checkbox
import ipywidgets as widgets
from __future__ import print_function
from IPython.display import display, clear_output
import matplotlib.dates as mdates
import matplotlib.ticker as ticker

## Get data

In [2]:
indices =  ['^AXJO','ES=F', 'VWRA.L', '^GSPC']


#indices =  ['ES=F', 'VWRA.L']


banks = ['CBA.AX', 'WBC.AX', 'ANZ.AX', 'NAB.AX']

ticker_code_list = indices
         
ticker_data = []
for ticker_code in ticker_code_list:
    ticker_obj = yf.Ticker(ticker_code)
    ticker_info = ticker_obj.info
    print(ticker_info)
    ticker_short_name = ticker_info.get('shortName', f'{ticker_code}')
    ticker_history = ticker_obj.history(period='14d',interval='15m')
    ticker_history_close = ticker_history['Close']
    
    ticker_history_close_normalised = ticker_history_close / ticker_history_close.mean()
    
    for i in range(len(ticker_history_close_normalised) - 1):
        if ticker_history_close_normalised.index[i + 1] - ticker_history_close_normalised.index[i] > timedelta(hours=2):
            ticker_history_close_normalised.loc[ticker_history_close_normalised.index[i + 1]] = None
    
        ticker_data_dict = {'Code': ticker_code,
                            'Name': ticker_short_name,
                            'Close': ticker_history_close,
                            'Close normalised': ticker_history_close_normalised
        }
    ticker_data.append((ticker_data_dict))


{'maxAge': 86400, 'priceHint': 2, 'previousClose': 8541.1, 'open': 8541.1, 'dayLow': 8536.7, 'dayHigh': 8613.0, 'regularMarketPreviousClose': 8541.1, 'regularMarketOpen': 8541.1, 'regularMarketDayLow': 8536.7, 'regularMarketDayHigh': 8613.0, 'volume': 0, 'regularMarketVolume': 0, 'averageVolume': 779185, 'averageVolume10days': 762540, 'averageDailyVolume10Day': 762540, 'bid': 0.0, 'ask': 0.0, 'bidSize': 0, 'askSize': 0, 'fiftyTwoWeekLow': 7169.2, 'fiftyTwoWeekHigh': 8639.1, 'fiftyDayAverage': 8346.752, 'twoHundredDayAverage': 8239.109, 'currency': 'AUD', 'tradeable': False, '52WeekChange': 10.351551, 'quoteType': 'INDEX', 'symbol': '^AXJO', 'language': 'en-US', 'region': 'US', 'typeDisp': 'Index', 'quoteSourceName': 'Delayed Quote', 'triggerable': False, 'customPriceAlertConfidence': 'LOW', 'exchange': 'ASX', 'messageBoardId': 'finmb_INDEXAXJO', 'exchangeTimezoneName': 'Australia/Sydney', 'exchangeTimezoneShortName': 'AEST', 'gmtOffSetMilliseconds': 36000000, 'market': 'au_market', 'es

## Plot data

In [3]:
exchange_tz = pytz.timezone('Australia/Sydney')
exchange_open_local = time(hour=10, minute=0, second=0)
exchange_close_local = time(hour=16, minute=0, second=0)
exchange_open_tz = exchange_tz.localize(datetime.combine(date.today(), exchange_open_local))
exchange_close_tz = exchange_tz.localize(datetime.combine(date.today(), exchange_close_local))

days = mdates.DayLocator()  # every day
daysFmt = mdates.DateFormatter('%a, %d-%m')
ticks = [mdates.date2num(date.today())]

fig, ax = plt.subplots(figsize=(19.2,10.8))
plt.close()
line_list = {}

for dataset in ticker_data:
    line_list[dataset['Name']] = ax.plot(dataset['Close normalised'].index,
                                         dataset['Close normalised'],
                                        label = dataset['Name'])
              
zoom = 1
pan = 0
        
now = datetime.now(timezone.utc)
time_range = timedelta(days = 5)
time_range_zoomed = time_range / zoom
margin = timedelta(hours = 4) / zoom
high = now + margin
low = now - time_range_zoomed - margin
offset = time_range * pan
high_set = high - offset
low_set = low - offset
print('Range: ' + str(time_range))
print('Pan: ' + str(offset))
ax.set_xlim (low_set, high_set)
ax.set_title('World indices')             

#Add on x axis date ticks
ax.xaxis.set_major_locator(days)
ax.xaxis.set_major_formatter(daysFmt)
ax.xaxis_date(tz=None)
ax.grid(visible=True, which='major', axis='both')
ax.legend(loc='upper right')



for days_offset in range(16):
    start_time = exchange_open_tz - timedelta(days=days_offset)
    end_time = exchange_close_tz - timedelta(days=days_offset)
    is_weekend = start_time.weekday() >= 5

    # blacken the nights
    ax.axvspan(end_time, start_time + timedelta(days=1), facecolor=(0.98, 0.98, 0.98))
    if is_weekend:
         ax.axvspan(start_time, end_time, facecolor=(0.95, 0.95, 0.95))

shift_factor_dict = {}
enabled_dict = {}
for dataset in ticker_data:
    shift_factor_dict[dataset['Name']] = 1.0
    enabled_dict[dataset['Name']] = True
    
    

def interactive_plot(zoom, pan, **kwargs):
    
    
    global time_range_zoomed
    global margin
    global ax
    global fig
    global now
    global time_range
    global high
    
    shift_factors = []
    enabled_flags = []
    for key, val in kwargs.items():
        if key.startswith('Enable'):
            enabled_flags.append(val)
        if key.startswith('Adjust'):
            shift_factors.append(val)

    time_range_zoomed = time_range / zoom
    margin = timedelta(hours = 4) / zoom
    low = now - time_range_zoomed - margin
    offset = time_range * pan
    high_set = high - offset
    low_set = low - offset
    ax.set_xlim (low_set, high_set)
    
    
    for dataset, shift_factor, enabled_flag in zip(ticker_data, shift_factors, enabled_flags):
        line = line_list[dataset['Name']][0]
        # line.set_xdata(dataset['Close normalised'].index)
        if enabled_flag:
            if shift_factor != shift_factor_dict[dataset['Name']] or enabled_flag != enabled_dict[dataset['Name']]:
                line.set_ydata(dataset['Close normalised'] * shift_factor)
                shift_factor_dict[dataset['Name']] = shift_factor
        else:
            line.set_ydata(None)
            
        enabled_dict[dataset['Name']] = enabled_flag
    
    return fig


keywords = {'zoom' : FloatSlider(min=1, max=5, step=0.0005, value=1),
            'pan': FloatSlider(min=0, max=1, step=0.001, value=0)}

widget_list = []
for ticker in ticker_data:
    keywords['Enable: ' + ticker['Code']] = Checkbox(True)
    keywords['Adjust: ' + ticker['Code']] = FloatSlider(min=0.98, max=1.02, step=0.0001, value=1)
    widget_list.append(keywords['Enable: ' + ticker['Code']])
    widget_list.append(keywords['Adjust: ' + ticker['Code']])
    widget_list.append(keywords['zoom'])
    widget_list.append(keywords['pan'])


_ = interact(interactive_plot, **keywords)



Range: 5 days, 0:00:00
Pan: 0:00:00


interactive(children=(FloatSlider(value=1.0, description='zoom', max=5.0, min=1.0, step=0.0005), FloatSlider(v…